In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk

class AnomalyDetectionGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Predictive Maintenance")
        
        self.file_path = None
        self.data = None
        self.anomalies = None
        self.maintenance_schedule = None
        
        # Define custom colors
        self.primary_color = "#3F51B5"  # Indigo
        self.secondary_color = "#E8EAF6"  # Light Indigo
        self.text_color = "#212121"  # Dark Grey
        
        # Create GUI elements with custom styles
        self.label = ttk.Label(root, text="Select dataset:", foreground=self.text_color)
        self.label.pack(pady=10)
        
        self.browse_button = ttk.Button(root, text="Browse", command=self.browse_file, style="Browse.TButton")
        self.browse_button.pack()
        
        self.detect_button = ttk.Button(root, text="Predictive Maintenance", command=self.detect_anomalies, style="Detect.TButton")
        self.detect_button.pack(pady=10)
        
        self.anomalies_label = ttk.Label(root, text="Maintenance Required Items:", foreground=self.text_color)
        self.anomalies_label.pack()
        
        self.anomalies_text = tk.Text(root, height=15, width=50, background=self.secondary_color, foreground=self.text_color)
        self.anomalies_text.pack()
        
        self.schedule_label = ttk.Label(root, text="Maintenance Schedule:", foreground=self.text_color)
        self.schedule_label.pack()
        
        self.schedule_text = tk.Text(root, height=15, width=50, background=self.secondary_color, foreground=self.text_color)
        self.schedule_text.pack()
        
        # Configure styles for buttons
        self.configure_styles()
        
        # Configure window fullscreen
        self.root.attributes('-fullscreen', True)
        self.root.bind('<Escape>', self.exit_fullscreen)
        
        # Bind window resize event
        self.root.bind('<Configure>', self.resize_text_fields)
        
    def browse_file(self):
        self.file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
        
    def detect_anomalies(self):
        if self.file_path is None:
            messagebox.showerror("Error", "Please select a CSV file.")
            return
        
        try:
            # Load the dataset
            self.data = pd.read_csv(self.file_path)
            
            # Select relevant features
            features = ['Temperature', 'Vibration', 'Pressure']
            X = self.data[features].values
            
            # Scale the data
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            
            # Train the Isolation Forest model
            model = IsolationForest(contamination=0.05)
            model.fit(X_scaled)
            
            # Predict the anomalies
            predictions = model.predict(X_scaled)
            
            # Identify the anomalous samples
            self.anomalies = self.data[predictions == -1]
            
            # Generate maintenance schedule
            self.generate_maintenance_schedule()
            
            # Display results in the GUI
            self.anomalies_text.delete(1.0, tk.END)
            self.anomalies_text.insert(tk.END, str(self.anomalies))
            
            self.schedule_text.delete(1.0, tk.END)
            for task in self.maintenance_schedule:
                self.schedule_text.insert(tk.END, str(task) + '\n')
                
        except Exception as e:
            messagebox.showerror("Error", str(e))
        
    def generate_maintenance_schedule(self):
        self.maintenance_schedule = []
        for index, anomaly in self.anomalies.iterrows():
            maintenance_task = {
                'Timestamp': anomaly['Timestamp'],
                'Equipment': anomaly['Equipment'],
                'Task': 'Routine Maintenance',
                'Priority': 'High'
            }
            self.maintenance_schedule.append(maintenance_task)
            
    def configure_styles(self):
        # Configure style for the "Browse" button
        style = ttk.Style()
        style.configure("Browse.TButton",
                        foreground=self.text_color,
                        background=self.primary_color,
                        font=('Arial', 12, 'bold'),
                        padding=8,
                        width=15)
        
        # Configure style for the "Predictive Maintenance" button
        style.configure("Detect.TButton",
                        foreground=self.text_color,
                        background=self.primary_color,
                        font=('Arial', 12, 'bold'),
                        padding=8,
                        width=20)
        
    def exit_fullscreen(self, event):
        self.root.attributes('-fullscreen', False)
        
    def resize_text_fields(self, event):
        if self.root.attributes('-fullscreen'):
            width = int(self.root.winfo_width() * 0.4)
            self.anomalies_text.configure(width=width)
            self.schedule_text.configure(width=width)

# Create the Tkinter application
root = tk.Tk()

# Customize the appearance of the root window
root.configure(background="#FAFAFA")  # Light Grey background color

app = AnomalyDetectionGUI(root)

# Run the Tkinter event loop
root.mainloop()
